In [5]:
import pandas as pd
from turpy.preprocess import TextPreprocesser
from turpy.models import TfIdfClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

In [24]:
# Read data
df = pd.read_csv("data/topics.csv")

X = df["text"]
y = df["category"]

# Preprocess text
preprocess = TextPreprocesser(
                                remove_diacritics=True, replace_stopwords=True
                            )

X = preprocess.transform(X)

# Train a TfIdf Classifier

model = TfIdfClassifier()
print(np.mean(cross_val_score(model, X, y, cv=5)))

0.636530612244898


In [25]:
# Sklearn tf-idf and without any preprocessing or augmentation

df = pd.read_csv("data/topics.csv")

X = df["text"]
y = df["category"]


model = TfIdfClassifier()
print(np.mean(cross_val_score(model, X, y, cv=5)))

0.6408163265306122
